In [1]:
import os
import seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, RegularPolygon
from matplotlib.path import Path
from matplotlib.projections.polar import PolarAxes
from matplotlib.projections import register_projection
from matplotlib.spines import Spine
from matplotlib.transforms import Affine2D
import matplotlib as mpl
import matplotlib.font_manager
mpl.use('Agg')
params = {'font.family': 'serif','font.serif': 'Times', 'text.usetex': True,'mathtext.fontset': 'custom'}
mpl.rcParams.update(params)

In [2]:
def parse_result_log(log):
    if os.path.exists(log):
        with open(log, 'r') as f:
            lines = f.readlines()
        lines = [lines[-2].strip(), lines[-1].strip()]
        lines = ' '.join(lines)
        print(lines)
        maps = lines.split('[')[1].split(']')[0].split(" ")
        maps = [float(m) for m in maps if m != '']
        mean_ap = np.mean(maps)
        return mean_ap
    else:
        raise FileNotFoundError(f"File {log} not found")

In [3]:
method_ruan_logfolder = '../outputs/ckpt_swallow_stage2_lgte'
method_hyder_logfolder = '../outputs/ckpt_swallow_2tower_10ep'
our_method_logfolder = '../outputs/2tower_crossmamba_3layer_ep30_vw0.7_heatmap_channelagg'

In [4]:
action_names = ['LaryngealVestibuleClosure', 'UESOpen', 'OralDelivery', 'ThroatTransport', 'HyoidExercise', 'ThroatSwallow', 'SoftPalateLift']

In [5]:
ori_ruan = [62.6, 65.4, 41.0, 59.4, 44.3, 19.7, 57.9]

In [6]:
N = len(action_names)
methods = ['Ruan et al.', 'Hyder et al.', 'Our method']
mean_aps = np.zeros((N, 3))
for i, action_name in enumerate(action_names):
    for j, method_logfolder in enumerate([method_ruan_logfolder, method_hyder_logfolder, our_method_logfolder]):
        log = os.path.join(method_logfolder, f'{action_name}.log')
        mean_aps[i, j] = parse_result_log(log)

mAP: [83.19794321 82.10801152 82.07432769 80.71208958 77.87450497 70.47973393 52.64406377]
mAP: [80.63689084 80.63689084 79.14625736 78.56582354 78.28230796 68.40265753 54.93958443]
mAP: [84.19917515 83.96611947 83.35091606 82.05253735 79.28553252 73.51354381 60.92700566]
mAP: [80.47257757 80.47257757 79.72972197 79.71596355 79.13463146 76.12276668 63.793633  ]
mAP: [80.653641   80.39421165 80.03991655 79.74699866 79.24959699 71.00106177 44.19740581]
mAP: [85.92743331 85.92743331 85.49861035 85.1551507  85.01894161 83.60160156 77.80422039]
mAP: [70.86388896 68.78206713 57.41976971 42.39032597 37.6922586  27.61980357 14.18004333]
mAP: [71.68415146 69.60929918 60.06079011 46.17794474 40.656449   29.09489665 13.68754816]
mAP: [73.62529893 69.67333077 62.66340268 51.12580904 39.45114863 21.50320215 12.22692871]
mAP: [84.16358258 83.68686947 82.18785068 80.35533246 76.81303038 71.63411781 41.70264217]
mAP: [86.019819   86.019819   85.47263413 84.26390764 83.36518398 68.78751952 52.42377541]

In [7]:
mean_aps

array([[75.5843821 , 74.37291607, 78.18497572],
       [77.06312454, 73.6118332 , 84.1333416 ],
       [45.56402247, 47.28158276, 47.18130299],
       [74.36334651, 78.05037981, 82.20825104],
       [48.24203376, 48.37633473, 56.01858296],
       [29.06654188, 21.04066974, 27.28729212],
       [63.72177472, 72.31435541, 75.29755152]])

In [8]:
ori_ruan = np.array(ori_ruan)
ori_ruan.T
mean_aps = np.concatenate([ori_ruan.T.reshape(-1, 1), mean_aps], axis=1)

In [9]:
mean_aps

array([[62.6       , 75.5843821 , 74.37291607, 78.18497572],
       [65.4       , 77.06312454, 73.6118332 , 84.1333416 ],
       [41.        , 45.56402247, 47.28158276, 47.18130299],
       [59.4       , 74.36334651, 78.05037981, 82.20825104],
       [44.3       , 48.24203376, 48.37633473, 56.01858296],
       [19.7       , 29.06654188, 21.04066974, 27.28729212],
       [57.9       , 63.72177472, 72.31435541, 75.29755152]])

In [10]:
axis_range = [[40, 80], [50, 90], [10, 50], [50, 90], [20, 60], [10, 30], [40, 80]]

In [11]:
# Normalize data for each action independently
def normalize_data(data):
    normalized_data = np.zeros_like(data)
    
    for i, (min_val, max_val) in enumerate(axis_range):
        normalized_data[i, :] = (data[i, :] - min_val) / (max_val - min_val)
    return normalized_data

In [12]:
normalized_mean_aps = normalize_data(mean_aps)

In [13]:
methods = ['Ruan et al.', 'Ruan et al. *', 'Hyder et al.', 'Our method']
df = pd.DataFrame(mean_aps, columns=methods, index=action_names)
df

,Ruan et al.,Ruan et al. *,Hyder et al.,Our method
LaryngealVestibuleClosure,62.6,75.584382,74.372916,78.184976
UESOpen,65.4,77.063125,73.611833,84.133342
OralDelivery,41.0,45.564022,47.281583,47.181303
ThroatTransport,59.4,74.363347,78.050380,82.208251
HyoidExercise,44.3,48.242034,48.376335,56.018583
ThroatSwallow,19.7,29.066542,21.040670,27.287292
SoftPalateLift,57.9,63.721775,72.314355,75.297552


In [14]:
normalized_mean_aps

array([[0.565     , 0.88960955, 0.8593229 , 0.95462439],
       [0.385     , 0.67657811, 0.59029583, 0.85333354],
       [0.775     , 0.88910056, 0.93203957, 0.92953257],
       [0.235     , 0.60908366, 0.7012595 , 0.80520628],
       [0.6075    , 0.70605084, 0.70940837, 0.90046457],
       [0.485     , 0.95332709, 0.55203349, 0.86436461],
       [0.4475    , 0.59304437, 0.80785889, 0.88243879]])

In [15]:
df.to_csv('mean_ap.csv')

In [16]:
def radar_factory(num_vars, frame='circle'):
    """
    Create a radar chart with `num_vars` axes.

    This function creates a RadarAxes projection and registers it.

    Parameters
    ----------
    num_vars : int
        Number of variables for radar chart.
    frame : {'circle', 'polygon'}
        Shape of frame surrounding axes.

    """
    # calculate evenly-spaced axis angles
    theta = np.linspace(0, 2*np.pi, num_vars, endpoint=False)

    class RadarTransform(PolarAxes.PolarTransform):

        def transform_path_non_affine(self, path):
            # Paths with non-unit interpolation steps correspond to gridlines,
            # in which case we force interpolation (to defeat PolarTransform's
            # autoconversion to circular arcs).
            if path._interpolation_steps > 1:
                path = path.interpolated(num_vars)
            return Path(self.transform(path.vertices), path.codes)

    class RadarAxes(PolarAxes):

        name = 'radar'
        PolarTransform = RadarTransform

        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            # rotate plot such that the first axis is at the top
            self.set_theta_zero_location('N')

        def fill(self, *args, closed=True, **kwargs):
            """Override fill so that line is closed by default"""
            return super().fill(closed=closed, *args, **kwargs)

        def plot(self, *args, **kwargs):
            """Override plot so that line is closed by default"""
            lines = super().plot(*args, **kwargs)
            for line in lines:
                self._close_line(line)

        def _close_line(self, line):
            x, y = line.get_data()
            # FIXME: markers at x[0], y[0] get doubled-up
            if x[0] != x[-1]:
                x = np.append(x, x[0])
                y = np.append(y, y[0])
                line.set_data(x, y)

        def set_varlabels(self, labels):
            self.set_thetagrids(np.degrees(theta), labels)

        def _gen_axes_patch(self):
            # The Axes patch must be centered at (0.5, 0.5) and of radius 0.5
            # in axes coordinates.
            if frame == 'circle':
                return Circle((0.5, 0.5), 0.5)
            elif frame == 'polygon':
                return RegularPolygon((0.5, 0.5), num_vars,
                                      radius=.5, edgecolor="k")
            else:
                raise ValueError("Unknown value for 'frame': %s" % frame)

        def _gen_axes_spines(self):
            if frame == 'circle':
                return super()._gen_axes_spines()
            elif frame == 'polygon':
                # spine_type must be 'left'/'right'/'top'/'bottom'/'circle'.
                spine = Spine(axes=self,
                              spine_type='circle',
                              path=Path.unit_regular_polygon(num_vars))
                # unit_regular_polygon gives a polygon of radius 1 centered at
                # (0, 0) but we want a polygon of radius 0.5 centered at (0.5,
                # 0.5) in axes coordinates.
                spine.set_transform(Affine2D().scale(.5).translate(.5, .5)
                                    + self.transAxes)
                return {'polar': spine}
            else:
                raise ValueError("Unknown value for 'frame': %s" % frame)

    register_projection(RadarAxes)
    return theta

In [33]:
# action_names = ['LaryngealVestibuleClosure', 'UESOpen', 'OralDelivery', 'ThroatTransport', 'HyoidExercise', 'ThroatSwallow', 'SoftPalateLift']
paper_action_names = [
    'Laryngeal\nVestibule\nClosure',
    'UES\nOpening',
    'Oral\nTransit',
    'Pharyngeal\nTransit',
    'Hyoid\nMotion',
    '\tSwallow\nInitiation',
    'Soft Palate\nElevation'
]
methods = ['Ruan et al. (A2Net)', 'Ruan et al. (ActionMamba) ', 'Hyder et al. (ActionMamba)', 'Ours (ActionMamba)']
# Plot radar chart
N = len(action_names)
theta = radar_factory(N, frame='polygon')
fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(projection='radar'))

# ax.set_title('Mean Average Precision', weight='bold', size='medium', position=(0.5, 1.1), 
#              horizontalalignment='center', verticalalignment='center')

grid_num = 4
# axis_range to grid_axis_range
grid_axis_range = []
for i, (min_val, max_val) in enumerate(axis_range):
    grid_axis_range.append(np.linspace(min_val, max_val, grid_num + 1))
grid_axis_range = np.array(grid_axis_range)
grid_axis_x = [0.2, 0.4, 0.6, 0.8]

colors = ['dodgerblue',  'lightgreen','gold', 'orangered']
for i in range(len(methods)):
    ax.plot(theta, normalized_mean_aps[:, i], color=colors[i], label=methods[i])
    ax.fill(theta, normalized_mean_aps[:, i], facecolor=colors[i], alpha=0.25)
    # label the axis with acutal values
    # for j in range(N):
    #     ax.text(theta[j], normalized_mean_aps[j, i], f"{mean_aps[j, i]:.1f}", color=colors[i], fontsize='small')
    # label the axis with grid axis range
    for j in range(N):
        ax.text(theta[j], grid_axis_x[i], f"{grid_axis_range[j, i]:.0f}", color='black', fontsize='medium')

# close the original axis ticks
ax.set_yticklabels([])
# set grid lines color
ax.yaxis.grid(True, color='grey', linestyle='-')
ax.xaxis.grid(True, color='grey', linestyle='-')
ax.set_varlabels(paper_action_names)
# increase the font size of the axis labels
ax.tick_params(axis='x', labelsize='xx-large')
# increase the distance between the axis labels and the plot
ax.tick_params(axis='x', pad=20)
# remove the outer border
ax.spines['polar'].set_visible(False)
legend = ax.legend(loc='upper right', bbox_to_anchor=(1.25, 0.2), labelspacing=0.1, fontsize='large')
plt.tight_layout()
plt.show()
plt.savefig('radar_chart.png', dpi=600)
plt.close()